In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns 
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.metrics import mean_squared_error
from hypopt import GridSearch
from sklearn.model_selection import cross_val_score,cross_val_predict, KFold,train_test_split,GridSearchCV,RandomizedSearchCV

In [2]:
na_value=['',' ','nan','Nan','NaN','na']
train=pd.read_csv('./train.csv',na_values=na_value)
store=pd.read_csv('./store.csv',na_values=na_value)
test=pd.read_csv('./test.csv',na_values=na_value)

C:\Users\Hp\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3072: DtypeWarning: Columns (7) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [3]:
train.head()

,Store,DayOfWeek,Date,Sales,Customers,Open,Promo,StateHoliday,SchoolHoliday
0,1,5,2015-07-31,5263,555,1,1,0,1
1,2,5,2015-07-31,6064,625,1,1,0,1
2,3,5,2015-07-31,8314,821,1,1,0,1
3,4,5,2015-07-31,13995,1498,1,1,0,1
4,5,5,2015-07-31,4822,559,1,1,0,1


In [4]:
store.head()

,Store,StoreType,Assortment,CompetitionDistance,CompetitionOpenSinceMonth,CompetitionOpenSinceYear,Promo2,Promo2SinceWeek,Promo2SinceYear,PromoInterval
0,1,c,a,1270.0,9.0,2008.0,0,NaN,NaN,NaN
1,2,a,a,570.0,11.0,2007.0,1,13.0,2010.0,"Jan,Apr,Jul,Oct"
2,3,a,a,14130.0,12.0,2006.0,1,14.0,2011.0,"Jan,Apr,Jul,Oct"
3,4,c,c,620.0,9.0,2009.0,0,NaN,NaN,NaN
4,5,a,a,29910.0,4.0,2015.0,0,NaN,NaN,NaN


In [5]:
test.head()

,Id,Store,DayOfWeek,Date,Open,Promo,StateHoliday,SchoolHoliday
0,1,1,4,2015-09-17,1.0,1,0,0
1,2,3,4,2015-09-17,1.0,1,0,0
2,3,7,4,2015-09-17,1.0,1,0,0
3,4,8,4,2015-09-17,1.0,1,0,0
4,5,9,4,2015-09-17,1.0,1,0,0


In [6]:
print("Shape of the Dataset:",train.shape)

Shape of the Dataset: (1017209, 9)


In [8]:
df_new = train.merge(store,on=["Store"], how="inner")
df_test=test.merge(store,on=["Store"], how="inner")
print(df_new.shape)

(1017209, 18)


In [120]:
print("Distinct number of Stores :", len(df_new["Store"].
unique()))
print("Distinct number of Days :", len(df_new["Date"].
unique()))
print("Average daily sales of all stores : ",round(df_new
["Sales"].mean(),2))

Distinct number of Stores : 1115
Distinct number of Days : 942
Average daily sales of all stores :  5773.82


In [121]:
df_new.dtypes

Store                          int64
DayOfWeek                      int64
Date                          object
Sales                          int64
Customers                      int64
Open                           int64
Promo                          int64
StateHoliday                  object
SchoolHoliday                  int64
StoreType                     object
Assortment                    object
CompetitionDistance          float64
CompetitionOpenSinceMonth    float64
CompetitionOpenSinceYear     float64
Promo2                         int64
Promo2SinceWeek              float64
Promo2SinceYear              float64
PromoInterval                 object
dtype: object

In [9]:
df_new.StateHoliday.replace({'0':0},inplace=True)
df_new.StateHoliday.unique()

array([0, 'a', 'b', 'c'], dtype=object)

working with time

In [10]:
df_new["DayOfWeek"].value_counts()


5    145845
4    145845
3    145665
2    145664
7    144730
6    144730
1    144730
Name: DayOfWeek, dtype: int64

In [11]:
df_new['Date'] = pd.to_datetime(df_new['Date'], infer_datetime_format=True)
df_new["Month"] = df_new["Date"].dt.month
df_new["Quarter"] = df_new["Date"].dt.quarter
df_new["Year"] = df_new["Date"].dt.year
df_new["Day"] = df_new["Date"].dt.day
df_new["Week"] = df_new["Date"].dt.week
df_new["Season"] = np.where(df_new["Month"].isin([3,4,5]),"Spring",
np.where(df_new["Month"].isin([6,7,8]),"Summer",np.where(df_new["Month"].isin
([9,10,11]),"Fall",np.where(df_new["Month"].isin([12,1,2]),"Winter","None"))))

In [12]:
df_test['Date'] = pd.to_datetime(df_test['Date'], infer_datetime_format=True)
df_test["Month"] = df_test["Date"].dt.month
df_test["Quarter"] = df_test["Date"].dt.quarter 
df_test["Year"] = df_test["Date"].dt.year
df_test["Day"] = df_test["Date"].dt.day
df_test["Week"] = df_test["Date"].dt.week
df_test["Season"] = np.where(df_test["Month"].isin([3,4,5]),"Spring",
np.where(df_test["Month"].isin([6,7,8]),"Summer",np.where(df_test["Month"].isin([9,10,11]),"Fall",np.where(df_test["Month"].isin([12,1,2]),"Winter","None"))))

In [126]:
df_test['Season'].value_counts()

Summer    26536
Fall      14552
Name: Season, dtype: int64

In [20]:
df_new['Season'].value_counts()

Spring    307740
Summer    262015
Winter    260904
Fall      186550
Name: Season, dtype: int64

In [13]:
df_new.isnull().sum()/df_new.shape[0] * 100

Store                         0.000000
DayOfWeek                     0.000000
Date                          0.000000
Sales                         0.000000
Customers                     0.000000
Open                          0.000000
Promo                         0.000000
StateHoliday                  0.000000
SchoolHoliday                 0.000000
StoreType                     0.000000
Assortment                    0.000000
CompetitionDistance           0.259730
CompetitionOpenSinceMonth    31.787764
CompetitionOpenSinceYear     31.787764
Promo2                        0.000000
Promo2SinceWeek              49.943620
Promo2SinceYear              49.943620
PromoInterval                49.943620
Month                         0.000000
Quarter                       0.000000
Year                          0.000000
Day                           0.000000
Week                          0.000000
Season                        0.000000
dtype: float64

In [14]:
#Replace nulls with the mode
df_new["CompetitionDistance"]=df_new["CompetitionDistance"].fillna(df_new["CompetitionDistance"].mode()[0])
#Double check if we still see nulls for the column
df_new["CompetitionDistance"].isnull().sum()/df_new.shape[0] * 100

0.0

In [15]:
target = ["Sales"]
numeric_columns = ["Customers","Open","Promo","Promo2",
"StateHoliday","SchoolHoliday","CompetitionDistance"]
categorical_columns = ["DayOfWeek","Quarter","Month","Year",
"StoreType","Assortment","Season"]

In [16]:
#Define a function that will intake the raw dataframe and thecolumn name and return a one hot encoded DF
def create_ohe(df, col):
    le = LabelEncoder() 
    a=le.fit_transform(df[col]).reshape(-1,1)
    ohe = OneHotEncoder(sparse=False)
    column_names = [col+ "_"+ str(i) for i in le.classes_]
    return(pd.DataFrame(ohe.fit_transform(a),columns =column_names))

In [17]:
#Since the above function converts the column, one at a time
#We create a loop to create the final dataset with all features
temp = df_new[numeric_columns]
for column in categorical_columns: 
    temp_df = create_ohe(df_new,column)
    temp = pd.concat([temp,temp_df],axis=1)

In [18]:
print("Shape of Data:",temp.shape)
print("Distinct Datatypes:",temp.dtypes.unique())

Shape of Data: (1017209, 44)
Distinct Datatypes: [dtype('int64') dtype('O') dtype('float64')]


In [19]:
temp['Sales']=df_new['Sales']
temp=temp.loc[(temp['Customers']<7000) & (temp['Sales']<=2000) ]

In [20]:
print(temp.columns[temp.dtypes=="object"])

Index(['StateHoliday'], dtype='object')


In [21]:
temp["StateHoliday"].unique()

array([0, 'a', 'b', 'c'], dtype=object)

stateholiday should have 0 or 1, to indicate if there is holiday or not

In [22]:
temp["StateHoliday"]= np.where(temp["StateHoliday"]== '0',0,1)
#One last check of the data type
temp.dtypes.unique()

array([dtype('int64'), dtype('int32'), dtype('float64')], dtype=object)

In [23]:
temp.head()

,Customers,Open,Promo,Promo2,StateHoliday,SchoolHoliday,CompetitionDistance,DayOfWeek_1,DayOfWeek_2,DayOfWeek_3,...,StoreType_c,StoreType_d,Assortment_a,Assortment_b,Assortment_c,Season_Fall,Season_Spring,Season_Summer,Season_Winter,Sales
5,0,0,0,0,1,0,1270.0,0.0,0.0,0.0,...,1.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0
12,0,0,0,0,1,0,1270.0,0.0,0.0,0.0,...,1.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0
19,0,0,0,0,1,0,1270.0,0.0,0.0,0.0,...,1.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0
26,0,0,0,0,1,0,1270.0,0.0,0.0,0.0,...,1.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0
33,0,0,0,0,1,0,1270.0,0.0,0.0,0.0,...,1.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0


In [24]:
np.random.seed(1234)
kfold = KFold(n_splits=5)
pipe= Pipeline([("scaler", StandardScaler()), ("forest", RandomForestRegressor(n_estimators=5))]) 
#using pipeline gave me a higher means squared error


In [227]:
scaler=StandardScaler()
X_train, X_test, y_train, y_test = train_test_split(temp,temp[target],test_size=0.2,random_state=1234)
scaler.fit(X_train)
X_train=scaler.transform(X_train) 
X_test=scaler.transform(X_test)

x_train, x_val, y_train, y_val = train_test_split(X_train,y_train,test_size=0.1,random_state=1234)

In [202]:
#Check the sizes of all newly created datasets
print("Shape of x_train:",x_train.shape)
print("Shape of x_val:",x_val.shape)
print("Shape of x_test:",X_test.shape)
print("Shape of y_train:",y_train.shape)
print("Shape of y_val:",y_val.shape)
print("Shape of y_test:",y_test.shape)

Shape of x_train: (128620, 45)
Shape of x_val: (14292, 45)
Shape of x_test: (35728, 45)
Shape of y_train: (128620, 1)
Shape of y_val: (14292, 1)
Shape of y_test: (35728, 1)


In [251]:
#score=cross_val_score(pipe,temp,df_new[target],n_jobs=-1,cv=kfold)
param={'n_estimators':range(1,200,10)}
#tree=RandomizedSearchCV(estimator=RandomForestRegressor(),param_distributions=param,iid=False, cv=kfold,random_state=1234,scoring='neg_mean_squared_error')
#tree.fit(x_train,y_train)
tree.score(x_val,y_val)

-0.1246983169236962

In [252]:
#pipe.score(x_train,y_train)
#np.sqrt(mean_squared_error(pipe.predict(X_test),y_test))
tree.best_params_



{'n_estimators': 31}

172871